In [4]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from matplotlib import 
#hdEsLYho81touZGfPyuD api key

In [97]:
#Import stock data from quandl
quandl.ApiConfig.api_key = "hdEsLYho81touZGfPyuD"
mydata = quandl.get("WIKI/AAPL")

print(mydata.tail())


              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2018-03-21  175.04  175.09  171.26  171.270  35247358.0          0.0   
2018-03-22  170.00  172.68  168.60  168.845  41051076.0          0.0   
2018-03-23  168.39  169.92  164.94  164.940  40248954.0          0.0   
2018-03-26  168.07  173.10  166.44  172.770  36272617.0          0.0   
2018-03-27  173.68  175.15  166.92  168.340  38962839.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2018-03-21          1.0     175.04     175.09    171.26     171.270   
2018-03-22          1.0     170.00     172.68    168.60     168.845   
2018-03-23          1.0     168.39     169.92    164.94     164.940   
2018-03-26          1.0     168.07     173.10    166.44     172.770   
2018-03-27          1.0     173.68     175.15    166.92     168.340  

In [98]:
#Get the adjusted close price
mydata = mydata[["Adj. Close"]]

print(mydata.tail())

            Adj. Close
Date                  
2018-03-21     171.270
2018-03-22     168.845
2018-03-23     164.940
2018-03-26     172.770
2018-03-27     168.340


In [99]:
days_out = 20 #days to predict out in the future
#create another column (the target or dependent variable) shifted n units up
mydata['Prediction'] = mydata[['Adj. Close']].shift(-days_out)
print(mydata.tail())

            Adj. Close  Prediction
Date                              
2018-03-21     171.270         NaN
2018-03-22     168.845         NaN
2018-03-23     164.940         NaN
2018-03-26     172.770         NaN
2018-03-27     168.340         NaN


In [100]:
#create the independent dataset (X)
#convert dataframe to a numpy array
x = np.array(mydata.drop(['Prediction'],1)) #drop the prediction column because we will not use it in training
#remove the last 'n' rows since the bottom n rows don't have a prediciton available.
x = x[:-days_out]

In [101]:
#create dependent data set (y)
#convert the dataframe to a numpy array (all of the values including NaN's)
y = np.array(mydata['Prediction'])
#get all of y values except last n rows 
y = y[:-days_out]
print(y)


[  0.44843584   0.45034721   0.45946295 ... 164.94       172.77
 168.34      ]


In [102]:
#Split data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [111]:
#create and train the support vector regressor
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

svr_rbf.fit(x_train,y_train)


SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [109]:
#Testing model: Score returns the coeficcient of determination R^2 of the prediction
#The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9950536325586973


In [112]:
#Create and train the linear regression model
lr = LinearRegression()
#Training
lr.fit(x_train, y_train) 

#test it
lr_confidence = lr.score(x_test, y_test)
print("linear regression confidence: ", lr_confidence)

linear regression confidence:  0.9949072317131588


In [113]:
x_forecast = np.array(mydata.drop(['Prediction'],1))[-days_out:]
print(x_forecast)

[[178.12 ]
 [175.   ]
 [176.21 ]
 [176.82 ]
 [176.67 ]
 [175.03 ]
 [176.94 ]
 [179.98 ]
 [181.72 ]
 [179.97 ]
 [178.44 ]
 [178.65 ]
 [178.02 ]
 [175.3  ]
 [175.24 ]
 [171.27 ]
 [168.845]
 [164.94 ]
 [172.77 ]
 [168.34 ]]


In [117]:
lr_prediction = lr.predict(x_forecast)
print("Linear prediction model for next n days:",lr_prediction)
# print(x_forecast['Prediction'])
svm_prediction = svr_rbf.predict(x_forecast)
print("suport vector regressor prediction model for next n days:",svm_prediction)


Linear prediction model for next n days: [180.59413338 177.43237866 178.658572   179.27673558 179.12472814
 177.46278015 179.39834153 182.47902561 184.2423119  182.46889179
 180.91841592 181.13122633 180.49279509 177.73639354 177.67559057
 173.65246037 171.19500678 167.23774647 175.17253475 170.68324841]
suport vector regressor prediction model for next n days: [172.0395054  168.87337521 169.74762385 170.67718732 170.4602275
 168.85456983 170.84081608 171.35051459 151.53806545 171.38943595
 172.31338361 172.48054508 171.95264412 168.80178465 168.79602625
 174.15281043 172.66731099 162.72482466 174.71752505 174.29910752]


In [120]:
#for plotting
import matplotlib.pyplot as plt
%matplotlib inline


plt.plot(mydata['Prediction'])
plt.plot(svm_prediction)
# plt.plot(valid[['Close','Predictions']])


ValueError: view limit minimum -36837.6 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units